In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample

from itertools import chain
from random import sample 
import scipy

### Read the Data

In [ ]:
data = pd.read_csv('findings_data.csv')

In [ ]:
data.head()

### Understand the Distribution
To understand distributions of variables as they relate to diseases, let's try splitting up the 'Finding Labels' column into one additional column per disease (e.g. one for 'Cardiomegaly', one for 'Emphysema', etc.) and put a binary flag in that column to indicate the presence of the disease

In [ ]:
all_labels = np.unique(list(chain(*data['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        data[c_label] = data['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
data.sample(3)

### TODO - 1
Now write your code here to visualize some of the distributions specified in the instructions

In [ ]:
# Write your code here

### How age & gender relate to the mass size?
Finally, look at if and how age & gender relate to mass size in individuals who have a mass as a finding:

In [ ]:
plt.scatter(data['Patient Age'],data['Mass_Size'])

### TODO - 2
Use the following to see if there are significant correlations between the two:

`scipy.stats.pearsonr(mass_sizes,ages)`<br>
`scipy.stats.ttest_ind(mass_sizes_male, mass_sizes_female)`

In [ ]:
# Write your code here